In [1]:
# writing a script to count barcodes in a fasta file

from glob import glob
from Bio import SeqIO
import pandas as pd

In [ ]:
# inputs:
# sample name
# fasta file


SAMPLE_TO_REF_SEQ_LEFT = {
    "2A": "GAACTATACAAATAA",
    "2B": "GAACTATACAAATAA",
    "2C": "GAACTATACAAATAA",
    "2D": "GAACTATACAAATAA",
    "3A": "ACTATATATAATCCAATC", #supposedul
    "3B": "GCTTGGTTTTGGAGA", 
    "3C": "TGAGTAGTACTGTGCGC",
}
SAMPLE_TO_REF_SEQ_RIGHT = {
    "2A": "CTCGAGGTTCGAGTA",
    "2B": "CTCGAGGTTCGAGTA",
    "2C": "CTCGAGGTTCGAGTA",
    "2D": "CTCGAGGTTCGAGTA",
    "3A": "AGATCGGAAGAGCGT", # supposedly
    "3B": "AGATCGGAAGAGCGTCGT",
    "3C": "AGATCGGAAGAGCGTCGT",
}


def count_barcodes(fastx_in, ref_seq_left, ref_seq_right, ftype=None, n_lim = 1000, look_only_5flank = False, len_bc = 15):
    """extract barcodes from each read in the FASTQ or FASTA file and returns df with columns 'bc' and 'bc_count'."""

    bc_to_count = {}  # dictionary mapping barcode to number of occurences
    bc_to_count["XXX"] = 0  # for reads where flanking sequences don't match
    c= 0
    for r in SeqIO.parse(fastx_in, ftype):
        c+=1
        if c>n_lim:
            break
        
        seq = r.seq
        #print('seq',seq)

        # in case the barcode is found on the other strand.
        seq_with_rc = seq + seq.reverse_complement()

        left_pos = seq_with_rc.find(ref_seq_left)
        right_pos = seq_with_rc.find(ref_seq_right)

        # looking for both 5' and 3' flank being right for samples 2
        if not look_only_5flank:
            if left_pos != -1 and right_pos != -1:
                bc_start = left_pos + len(ref_seq_left)  # index of first letter of bc
                bc_end = right_pos  # +1 of index of last letter of bc
                # if 13 <= (bc_end-bc_start) <= 17:
                bc = ''.join(seq_with_rc[bc_start:bc_end])
                # assert 13 <= len(bc) <= 17, f'bc of length {len(bc)} not reasonable'
                if bc not in bc_to_count:
                    bc_to_count[bc] = 1
                else:
                    bc_to_count[bc] += 1
            else:
                
                bc_to_count["XXX"] += 1

        # if only looking for the 5' flank, ie. for full length samples 3
        # can fetch some examples later to demonstrate this in the 3 samples.
        else:
            if left_pos == -1:
                bc_to_count["XXX"] += 1
            else:
                bc_start = left_pos + len(ref_seq_left)
                bc_end = bc_start + len_bc
                bc = ''.join(seq_with_rc[bc_start:bc_end])
                if bc not in bc_to_count:
                    bc_to_count[bc] = 1
                else:
                    bc_to_count[bc] += 1
    df = pd.DataFrame.from_dict(bc_to_count, orient="index").reset_index()
    df.columns = ["bc", "bc_count"]
    print('counted',c,'reads')
    display(df)

    return df


In [4]:
# for 2A
illumina_dir = "/data/davidding/dms/illumina_data/missing2a/"

fa_files = [f for f in glob(illumina_dir+'**/*extendedFrags_filtered.fasta', recursive=True)]
f_to_path = dict([(f.split('/')[-1], f) for f in fa_files])
print(len(f_to_path))

# create bc df for each fasta file
for f in sorted(f_to_path.keys()):
    print(f, f_to_path[f])
    f_path = f_to_path[f]
    
    df_bc = count_barcodes(
        f_path, 
        SAMPLE_TO_REF_SEQ_LEFT['2A'], 
        SAMPLE_TO_REF_SEQ_RIGHT['2A'], 
        ftype='fasta', 
        n_lim=10000000000, 
        look_only_5flank =False)
    df_bc.to_csv(f_path.replace('.fasta', '_bc.csv'), index=False)



6
rice2A1_RNA_S1_L001_.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/missing2a/rice/2A/rice2A1_RNA_S1_L001_.extendedFrags_filtered.fasta
counted 3476590 reads


,bc,bc_count
0,XXX,539891
1,TCTTGTTACAACCAT,25
2,TTTTTTGTGGCAATC,1479
3,GGGCTGATCCGAGTT,153
4,CGCAAGCTTTGATTG,2
...,...,...
84928,CTTCAATGTCCTGAT,1
84929,CTTCCTCCCTCAGTT,1
84930,TAAAATCGTTGTTTT,1
84931,GAGAAACTTATTAAT,1


rice2A2_RNA_S2_L001_.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/missing2a/rice/2A/rice2A2_RNA_S2_L001_.extendedFrags_filtered.fasta
counted 4980317 reads


,bc,bc_count
0,XXX,774349
1,TTTTCGTGAGAGCCA,114
2,ATCGACATGGAGGGC,53
3,GTAGATTATCTTTAC,76
4,GTATTCTCACTTTAT,86
...,...,...
93000,TATTGGACGTCACTT,1
93001,ATTCATCACGTTTTA,1
93002,CTGGAACTTATTGAG,1
93003,ATGGGATGCTTTTAC,1


rice2A_DNA_S3_L001_.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/missing2a/rice/2A/rice2A_DNA_S3_L001_.extendedFrags_filtered.fasta
counted 4469257 reads


...

,bc,bc_count
0,XXX,1010581
1,TGTTTTGAAAGAAGA,45
2,TTAAAATGGACTTAT,85
3,TTCTCGCTCTATGGT,38
4,GTTTTTATTATAAAT,121
...,...,...
102887,GGTCAATTCCGCTAA,1
102888,TTATCGGCTGAAAAC,1
102889,GCAGCATGTAACTAG,1
102890,TATATCAATGTGTTT,1


sorghum2A2_RNA_S5_L001_.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/missing2a/sorghum/2A/sorghum2A2_RNA_S5_L001_.extendedFrags_filtered.fasta
counted 3593664 reads


,bc,bc_count
0,XXX,572322
1,TACAATTTTCTAAAT,58
2,TAATTGCCACATTAT,11
3,TTCAGTCAAAATTGG,7
4,GTAGCGATCTTCTCG,37
...,...,...
90879,GTTGGGGGAGGTCTT,1
90880,AACTTTTTACGTGTT,1
90881,TTCATATACAAATGC,1
90882,AAGTTCATGGATTGG,1


sorghum2A_DNA_S6_L001_.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/missing2a/sorghum/2A/sorghum2A_DNA_S6_L001_.extendedFrags_filtered.fasta
counted 7852720 reads


,bc,bc_count
0,XXX,1194419
1,TACTATTCTGTTTTT,250
2,TTGCCCCTGCGTGAT,157
3,GAATTAATGTCGTCT,174
4,ATCGCTGTGATAACC,41
...,...,...
97184,CAAATCTCTTTAGAA,1
97185,GGTACACCGTGCTTC,1
97186,ACTCCTGCCTATTGG,1
97187,CCATTTGCTCAACTT,1


In [68]:
# for all samplles apart from 2A
# find all fasta files
illumina_dir = '/data/davidding/dms/illumina_data/PROTOPLASTSUMMER24-425876467/BCLConvert_07_28_2024_02_06_09Z-753277525/'

fa_files = [f for f in glob(illumina_dir+'**/*extendedFrags_filtered.fasta', recursive=True)]
f_to_path = dict([(f.split('/')[-1], f) for f in fa_files])
print(len(f_to_path))

# create bc df for each fasta file
for f in sorted(f_to_path.keys()):
    print(f, f_to_path[f])
    f_path = f_to_path[f]
    

    sample_name = f_path.split('/')[-1][:2].upper()
    print(sample_name)

    if sample_name.startswith('3'):
        look_only_5flank = True
    else:
        look_only_5flank = False

    df_bc = count_barcodes(
        f_path, 
        SAMPLE_TO_REF_SEQ_LEFT[sample_name], 
        SAMPLE_TO_REF_SEQ_RIGHT[sample_name], 
        ftype='fasta', 
        n_lim=10000000000, 
        look_only_5flank =look_only_5flank)
    df_bc.to_csv(f_path.replace('.fasta', '_bc.csv'), index=False)





32
2b_dna_rep12_sorghum.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/PROTOPLASTSUMMER24-425876467/BCLConvert_07_28_2024_02_06_09Z-753277525/EG_protoplast_3-ds.9df57b00ae2a48c1a756dff099b4a610/2b_dna_rep12_sorghum.extendedFrags_filtered.fasta
2B
counted 9213455 reads


,bc,bc_count
0,XXX,1231698
1,TCATGCAATGGTTTA,3194
2,TGAAGTCGCGGAGGT,12221
3,ATATTAACGGATAGA,32757
4,ACCGTGAGATTTTTT,63
...,...,...
85531,CCGCAATCATTCCAC,1
85532,TCAAGTCAAACTTGG,1
85533,GTTGATACATTTTCA,1
85534,ATACATACGACTCTG,1


2b_dna_rep34_sorghum.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/PROTOPLASTSUMMER24-425876467/BCLConvert_07_28_2024_02_06_09Z-753277525/EG_protoplast_12-ds.0b361dc0124b4d378f01a022858a9df5/2b_dna_rep34_sorghum.extendedFrags_filtered.fasta
2B
counted 28270633 reads


,bc,bc_count
0,XXX,3774655
1,GTGCTGACGGGTGTG,64432
2,CTATATTAACTCTTTA,622
3,TTATCAAATTTTCTT,92
4,GGCGTATCTGGAGTG,44807
...,...,...
115416,TTCTGTTATCAAATAA,1
115417,TAAATATAACATATC,1
115418,ACATCTAGTTGCTTG,2
115419,AATATTGCTGGTTTT,1


2b_rep1_sorghum.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/PROTOPLASTSUMMER24-425876467/BCLConvert_07_28_2024_02_06_09Z-753277525/EG_protoplast_1-ds.a852c77554964539afce4473a203c368/2b_rep1_sorghum.extendedFrags_filtered.fasta
2B
counted 5433470 reads


...

,bc,bc_count
0,XXX,10364582
1,GATTCTTCTTTATAA,3
2,GGAGAGGTATAAACC,1
3,GTATAACTACTGAAA,1
4,TGCGTAGGTATGAAC,1
...,...,...
161190,GGGTATAAACTGCAT,1
161191,GGTGGCTGTACAAAT,1
161192,TTTAACGCTTCTATC,1
161193,AAATTATGGCCATAA,1


3c_rep2_sorghum.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/PROTOPLASTSUMMER24-425876467/BCLConvert_07_28_2024_02_06_09Z-753277525/EG_protoplast_20-ds.0a2ba1e8bc0241508806d879e68a9b12/3c_rep2_sorghum.extendedFrags_filtered.fasta
3C
counted 43181408 reads


,bc,bc_count
0,XXX,9074038
1,GCTGCTTGCCAAGAA,1
2,TCAGTAGGTCAACCA,1
3,GAGTAGTGCGTCGAC,226
4,TGCGCTATTCAATAA,1
...,...,...
171354,ATCCATGTATTAGAA,1
171355,TATTTCTTTGGATAT,1
171356,TGACTATAGTTGTAC,1
171357,TTCAGCAGACCATGC,1


3c_rep3_sorghum.extendedFrags_filtered.fasta /data/davidding/dms/illumina_data/PROTOPLASTSUMMER24-425876467/BCLConvert_07_28_2024_02_06_09Z-753277525/EG_protoplast_24-ds.d0c5a57bc091412f999121d1f5e066cf/3c_rep3_sorghum.extendedFrags_filtered.fasta
3C
counted 36871291 reads


,bc,bc_count
0,XXX,8028994
1,TCGAGCAGGAATGCA,1
2,GTTCTAGTGTGGGAC,1227
3,GACGGAATGCACAAA,2
4,ACGGATCGACGTTAA,2
...,...,...
136679,GTTCTTAAGGTTGGG,1
136680,GATCATAGTGGATCC,1
136681,ACCGAATCAAAAGAC,1
136682,GCTTTAATAAATGAT,1
